In [ ]:
! pip install pytorch-lightning~=1.5.0
! pip install datasets~=2.1.0
! pip install transformers
! pip install torchmetrics~=0.7.0
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.2/398.2 kB 29.6 MB/s eta 0:00:00
  Attempting uninstall: torchmetrics
    Found existing ins

In [ ]:
from transformers import AutoTokenizer, AutoModel, pipeline
from datasets import load_dataset
from datasets import load_metric
import torch
import torch.nn as nn
from transformers import RobertaTokenizerFast,RobertaForSequenceClassification, AdamW, TrainingArguments, Trainer,AutoModelForSequenceClassification
dataset = load_dataset("McGill-NLP/FaithDial")
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 512)
itemsUnique=[]
for items in dataset["train"]["BEGIN"]:
    for val in items:
        if val not in itemsUnique:
            itemsUnique.append(val)

id2label = {idx:itemsUnique for idx, itemsUnique in enumerate(itemsUnique)}
label2id = {itemsUnique:idx for idx, itemsUnique in enumerate(itemsUnique)}
print(id2label)

binary_dataset = dataset.map(lambda examples: {"original_response": examples["response"]  if examples["original_response"] == None else examples["original_response"]})
import numpy as np
def preprocess_function(examples):
    encoding=tokenizer(examples["knowledge"], examples["original_response"], truncation=True)
    tempLables=examples["BEGIN"]
    lables=np.zeros(len(label2id))
    for tempLable in tempLables:
        index=label2id[tempLable]
        lables[index]=1
    encoding["labels"]=lables
    
    return encoding
encoded_dataset = binary_dataset.map(preprocess_function,remove_columns=binary_dataset['train'].column_names)
print(dataset)

Extracting data files:   0%|          | 0/7 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/3539 [00:00<?, ? examples/s]

Generating test_random_split split:   0%|          | 0/1716 [00:00<?, ? examples/s]

Generating test_topic_split split:   0%|          | 0/1823 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/18357 [00:00<?, ? examples/s]

...

  0%|          | 0/18357 [00:00<?, ?ex/s]

  0%|          | 0/3417 [00:00<?, ?ex/s]

  0%|          | 0/1666 [00:00<?, ?ex/s]

  0%|          | 0/1751 [00:00<?, ?ex/s]

DatasetDict({
    test: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 3539
    })
    test_random_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1716
    })
    test_topic_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1823
    })
    train: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 18357
    })
    validation: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 3417
    })
    valid_random_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1666
    

In [ ]:
print(dataset["train"][1])
print(encoded_dataset["train"][1])

{'dialog_idx': 0, 'response': 'Well, I know that more people started using it after some restrictions on internet use were lifited in 1995', 'original_response': 'It used to be restricted, but around 1995, the restricted were lifted and commercial use of it began', 'history': ['Can you imagine the world without internet access?', 'Yeah, but once the access to the internet was a rare thing. do you remember?', 'I do. What else can you tell me ?'], 'knowledge': 'Use by a wider audience only came in 1995 when restrictions on the use of the Internet to carry commercial traffic were lifted.', 'BEGIN': ['Hallucination', 'Entailment'], 'VRM': ['Edification']}
{'input_ids': [0, 34447, 30, 10, 6012, 2437, 129, 376, 11, 7969, 77, 5165, 15, 5, 304, 9, 5, 3742, 7, 2324, 1861, 1703, 58, 4639, 4, 2, 2, 243, 341, 7, 28, 9393, 6, 53, 198, 7969, 6, 5, 9393, 58, 4639, 8, 1861, 304, 9, 24, 880, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label2id),label2id=label2id,id2label=id2label)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [ ]:
print(model)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
#     warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
#     logging_steps=1000,
    evaluation_strategy='steps',
#     eval_steps=1000,
    save_total_limit=1,
#     load_best_model_at_end=True,
#     metric_for_best_model='accuracy'
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
#     compute_metrics=load_metric('accuracy')
)

# Fine-tune the model
# trainer.train()

# Evaluate the model on the test set
results = trainer.evaluate(encoded_dataset['test'])
print(results)

{'eval_loss': 0.2807147800922394, 'eval_runtime': 19.1316, 'eval_samples_per_second': 184.982, 'eval_steps_per_second': 2.927}


In [ ]:
print(encoded_dataset)
encoded_dataset_test = encoded_dataset["test"]
print(encoded_dataset_test)
encoded_dataset_test_final = encoded_dataset_test.map(remove_columns="labels")
print(encoded_dataset_test_final)

predictions=trainer.predict(encoded_dataset_test_final)


DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3539
    })
    test_random_split: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1716
    })
    test_topic_split: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1823
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 18357
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3417
    })
    valid_random_split: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1666
    })
    valid_topic_split: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1751
    })
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3539
})


  0%|          | 0/3539 [00:00<?, ?ex/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3539
})


In [ ]:
print(predictions)

PredictionOutput(predictions=array([[ 6.3917584 ,  0.18404748, -6.9030695 , -6.8952913 ],
       [-3.9597135 ,  4.1742043 , -0.04775586, -4.4452944 ],
       [-2.7007527 ,  4.295292  , -1.0180308 , -5.4151883 ],
       ...,
       [ 6.317063  ,  0.75067997, -6.999736  , -7.154722  ],
       [ 5.5415106 ,  2.4479766 , -7.0483336 , -7.6445875 ],
       [ 5.229318  ,  4.2027454 , -6.4898705 , -7.152149  ]],
      dtype=float32), label_ids=None, metrics={'test_runtime': 19.0762, 'test_samples_per_second': 185.519, 'test_steps_per_second': 2.936})


In [ ]:
predictionsOp=trainer.predict(encoded_dataset['test'])
print(predictionsOp)

PredictionOutput(predictions=array([[ 6.3917584 ,  0.18404748, -6.9030695 , -6.8952913 ],
       [-3.9597135 ,  4.1742043 , -0.04775586, -4.4452944 ],
       [-2.7007527 ,  4.295292  , -1.0180308 , -5.4151883 ],
       ...,
       [ 6.317063  ,  0.75067997, -6.999736  , -7.154722  ],
       [ 5.5415106 ,  2.4479766 , -7.0483336 , -7.6445875 ],
       [ 5.229318  ,  4.2027454 , -6.4898705 , -7.152149  ]],
      dtype=float32), label_ids=array([[1., 0., 0., 0.],
       [0., 1., 1., 0.],
       [0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32), metrics={'test_loss': 0.2807147800922394, 'test_runtime': 20.4289, 'test_samples_per_second': 173.235, 'test_steps_per_second': 2.741})


In [ ]:
for predictionOp in predictionsOp.predictions:
  m = nn.Sigmoid()
  t = torch.from_numpy(predictionOp)
  output = m(t)
  print(output)

tensor([0.9983, 0.5459, 0.0010, 0.0010])
tensor([0.0187, 0.9848, 0.4881, 0.0116])
tensor([0.0629, 0.9866, 0.2654, 0.0044])
tensor([0.4358, 0.9871, 0.0344, 0.0010])
tensor([9.9486e-01, 9.8565e-01, 1.6462e-03, 8.7957e-04])
tensor([9.9179e-01, 9.8787e-01, 1.6310e-03, 5.9848e-04])
tensor([9.8746e-01, 9.9125e-01, 2.0496e-03, 6.3143e-04])
tensor([9.8241e-01, 9.8890e-01, 2.0341e-03, 4.3276e-04])
tensor([9.9204e-01, 9.8781e-01, 1.5887e-03, 6.0792e-04])
tensor([9.8525e-01, 9.9218e-01, 2.2500e-03, 6.9223e-04])
tensor([0.3120, 0.9923, 0.0464, 0.0011])
tensor([0.1618, 0.9499, 0.1401, 0.0074])
tensor([8.0389e-01, 9.7459e-01, 1.0667e-02, 6.7131e-04])
tensor([0.0237, 0.9863, 0.4003, 0.0086])
tensor([0.0724, 0.9908, 0.1262, 0.0030])
tensor([0.0262, 0.9855, 0.3675, 0.0083])
tensor([6.9839e-01, 9.8537e-01, 1.3691e-02, 6.5316e-04])
tensor([0.3304, 0.9918, 0.0290, 0.0010])
tensor([0.0260, 0.9879, 0.4049, 0.0075])
tensor([9.9038e-01, 9.9116e-01, 2.4579e-03, 9.6330e-04])
tensor([9.7465e-01, 9.8523e-01, 2.27

In [ ]:
prediction_converted=[]
for prediction in predictionsOp.predictions:
  indiarray=[]
  for indi in prediction:
    if indi>1:
      indiarray.append(1.0)
    else:
      indiarray.append(0.0)
  prediction_converted.append(indiarray)
print(prediction_converted)

[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0

In [ ]:
print(encoded_dataset['test']["labels"])

[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 1.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0], [1.0, 1.0

In [ ]:
from sklearn.metrics import f1_score
f1_score(encoded_dataset['test']["labels"], prediction_converted, average='macro')

0.47573792972809015

In [ ]:
count=0
total=0
for i in range(0,len(prediction_converted)):
  if prediction_converted[i]==encoded_dataset['test']["labels"][i]:
    count+=1
  total+=1
print("correct count",count)
print("total count",total)


correct count 2351
total count 3539


In [ ]:
print("accuracy",count/total)

accuracy 0.664311952528963


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = F"/content/gdrive/My Drive/Models/begin_classification_roberta_base.pt"

In [ ]:
# torch.save(model, path)

In [ ]:
model = torch.load(path)
print(model)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
print(label2id)

{'Hallucination': 0, 'Entailment': 1, 'Uncooperative': 2, 'Generic': 3}


NameError: ignored